## Geocode Land Use Covenants
Create a new file with lat/long for land use covenants

In [1]:
import requests
import pandas as pd
import time

## Geocode Data
Run code to get lat/long for datasets.

In [3]:
# Load the datasets
land_use_covenants = pd.read_csv('Land Use Covenants 2006-2022 (Controller and ED1).csv')

In [5]:
# Function to apply geocoding to land_use_covenants DataFrame
def apply_google_geocoding(df, address_column):
    df[["LAT", "LONG"]] = df[address_column].apply(google_geocode_address)
    return df


In [6]:
# Ensure the DataFrame exists before geocoding
print("Before Geocoding:")
print(land_use_covenants.head())  # Check the first few rows

Before Geocoding:
                                             Address  Zip Code  CD#  \
0           4660 N KESTER AVE Sherman Oaks, CA 91403   91403.0    4   
1  6938 N Laurel Canyon BLVD North Hollywood, CA ...   91605.0    2   
2      4520 S FIGUEROA ST 1-21 Los Angeles, CA 90037   90037.0    9   
3             7601 N HOLLYWOOD WAY BURBANK, CA 91505   91505.0    2   
4        1421 W TEMPLE ST 1-52 Los Angeles, CA 90026   90026.0    1   

   Year of Covenant  Affordable Units  Total Units % of Affordable  \
0              2007               NaN          NaN             NaN   
1              2007               NaN          NaN             NaN   
2              2007               NaN         21.0             NaN   
3              2007               NaN         20.0             NaN   
4              2007               NaN         52.0             NaN   

  Entitlements Type2 AH Studio Unit # AH 1BR Unit # AH 2BR Unit #  \
0          NaN   NaN              NaN           NaN           NaN

In [8]:
# clean address data
import re

def clean_address(address):
    """Removes unit numbers, ranges, and unnecessary text after the street name."""
    address = re.sub(r"\s\d+(-\d+)?\b", "", address)  # Removes numbers like "1-5"
    return address.strip()

# Apply the cleaning function before geocoding
land_use_covenants["Cleaned_Address"] = land_use_covenants["Address"].apply(clean_address)

# Check if the cleaning worked
print(land_use_covenants[["Address", "Cleaned_Address"]].head(10))

                                             Address  \
0           4660 N KESTER AVE Sherman Oaks, CA 91403   
1  6938 N Laurel Canyon BLVD North Hollywood, CA ...   
2      4520 S FIGUEROA ST 1-21 Los Angeles, CA 90037   
3             7601 N HOLLYWOOD WAY BURBANK, CA 91505   
4        1421 W TEMPLE ST 1-52 Los Angeles, CA 90026   
5          1117 S WESTLAKE AVE Los Angeles, CA 90006   
6        225 N AVENUE 25 1-146 Los Angeles, CA 90031   
7            7551 N JORDAN AVE Canoga Park, CA 91303   
8           7511 S MAIN ST 1-9 LOS ANGELES, CA 90003   
9      3325 S CANFIELD AVE 1-3 Los Angeles, CA 90034   

                                 Cleaned_Address  
0             4660 N KESTER AVE Sherman Oaks, CA  
1  6938 N Laurel Canyon BLVD North Hollywood, CA  
2             4520 S FIGUEROA ST Los Angeles, CA  
3               7601 N HOLLYWOOD WAY BURBANK, CA  
4               1421 W TEMPLE ST Los Angeles, CA  
5            1117 S WESTLAKE AVE Los Angeles, CA  
6                   225 N 

In [9]:
def google_geocode_address(address):
    if pd.isna(address) or address.strip() == "":
        return pd.Series([None, None])

    # Append city and state if missing
    if "Los Angeles" not in address:
        address += ", Los Angeles, CA"

    base_url = "https://maps.googleapis.com/maps/api/geocode/json"
    params = {
        "address": address,
        "key": API_KEY
    }
    try:
        response = requests.get(base_url, params=params)
        data = response.json()
        if data["status"] == "OK":
            location = data["results"][0]["geometry"]["location"]
            return pd.Series([location["lat"], location["lng"]])
        else:
            print(f"Geocoding failed for {address}: {data['status']}")
            return pd.Series([None, None])
    except Exception as e:
        print(f"Error geocoding {address}: {e}")
        return pd.Series([None, None])

# Apply geocoding to cleaned addresses
land_use_covenants = apply_google_geocoding(land_use_covenants, "Cleaned_Address")


Geocoding failed for 316 S CATALINA ST, Los Angeles, CA: UNKNOWN_ERROR


In [10]:
# Save geocoded datasets
land_use_covenants.to_csv('geocoded_land_use_covenants.csv', index=False)

In [ ]:
# Load geocoded data
land_use_covenants = pd.read_csv('geocoded_land_use_covenants.csv')
updated_rso_data = pd.read_csv('geocoded_updated_rso_data.csv')

In [ ]:
# Drop rows where Latitude or Longitude is missing
land_use_covenants = land_use_covenants.dropna(subset=['Latitude', 'Longitude'])

print(f"Remaining Land Use Covenant records: {len(land_use_covenants)}")

In [ ]:
# Display the first few rows of land use covenants
land_use_covenants.head()